In [138]:
import pandas as pd
pd.set_option('display.max_columns', None)

# 1. Define functions

## 1.1 Insert statistic being recorded as a Series in each table

In [139]:
def add_stat(frame):
    # Create new column to indicate statistic for each estimation 
    counter = 1
    while counter < len(frame.columns):
        if not (pd.isnull(frame.columns[counter])):
            # Keeps track of Stat column positiions
            pos_list.append(counter)
            frame.insert(loc=counter, column=counter, value=frame.columns[counter])
            counter += 1
        counter +=1
    frame.iloc[0,0] = "Country code"
    frame.columns = frame.iloc[0]
    frame.drop(frame.index[0], inplace=True)
    return frame

## 1.2 Melt each statistic based on year and country

In [140]:
def combine_subset(frame):
    # Slicing the Country code column
    x = frame.iloc[:,0]

    # Slicing each set of statistic to melt by year
    for i in range(len(pos_list)):
        current_item = pos_list[i]
        # Checks for position of last stat
        # If it is, use rest of table
        if pos_list[i] == pos_list[-1]:
            y = frame.iloc[:, current_item:]
        # If not, take years between stats
        else:
            next_item = pos_list[i + 1]
            y = frame.iloc[:, current_item:next_item]
        to_melt = pd.concat([x, y], axis=1)
        to_melt = to_melt.melt(id_vars=[to_melt.columns[0], to_melt.columns[1]], var_name='Year', value_name='Estimate')
        to_melt.columns = ['Country code','Statistic','Year','Estimate']
        to_melt['Estimate'] = to_melt['Estimate'].astype('float64').round(decimals=2)
        to_melt['Year'] = to_melt['Year'].apply(lambda x: str(int(x)) if isinstance(x, float) else str(x))
        to_melt['Country code'] = to_melt['Country code'].apply(lambda x: str(int(x)))
        sub_list.append(to_melt)

In [141]:
# Function to identify based on Country code
def add_class(x):
    x = int(x)
    if x in ca_list:
        return 'Country or area'
    elif x in ma_list:
        return 'Major area'
    elif x in r_list:
        return 'Region'
    else:
        # Aggregate designates specific regions not just based on geographical location
        return 'Aggregate'

# 2. Read UN spreadsheet.

## 2.1 Create annex for cross-referencing

In [142]:
# Slice common data from table
annex = pd.read_excel("UN_MigrantStockTotal_2015.xlsx", sheet_name=1, index_col=None, header=0, skiprows=13)
annex = annex.iloc[:,:5]
# Set first row as columns
annex.columns = annex.iloc[0]
# Drop first 2 columns
annex.drop([0,1],inplace=True)
annex.reset_index(drop=True, inplace=True)
annex = annex.fillna(0)

In [143]:
# Table that groups countries and areas by major area and region
groups = pd.read_excel("UN_MigrantStockTotal_2015.xlsx", sheet_name=7, index_col=None, header=0, skiprows=13)
groups.columns = groups.iloc[0]
groups = groups.drop(0)
groups = groups.reset_index(drop=True)
groups = groups.iloc[:,[0,1,3,4,6,7,9,10,11]]

# Add classification column in Annex to normalize
ca_list = groups.iloc[:,0].tolist()
ma_list = groups.iloc[:,3].unique().tolist()
r_list = groups.iloc[:,5].unique().tolist()

In [144]:
annex['Classification'] = annex['Country code']
annex['Classification'] = annex['Classification'].apply(add_class)

## Annex: Main reference for all countries or areas, major areas and regions.

In [145]:
annex.head(15)

0   Sort\norder Major area, region, country or area of destination Notes  \
0             1                                              WORLD     0   
1             2                                  Developed regions   (b)   
2             3                                 Developing regions   (c)   
3             4                          Least developed countries   (d)   
4             5  Less developed regions excluding least develop...     0   
5             6                                 Sub-Saharan Africa   (e)   
6             7                                             Africa     0   
7             8                                     Eastern Africa     0   
8             9                                            Burundi     0   
9            10                                            Comoros     0   
10           11                                           Djibouti     0   
11           12                                            Eritrea     0   
12           13                                           Ethiopia     0   
13           14                                              Kenya     0   
14           15                                         Madagascar     0   

0   Country code Type of data (a)   Classification  
0            900                0        Aggregate  
1            901                0        Aggregate  
2            902                0        Aggregate  
3            941                0        Aggregate  
4            934                0        Aggregate  
5            947                0        Aggregate  
6            903                0       Major area  
7            910                0           Region  
8            108              B R  Country or area  
9            174                B  Country or area  
10           262              B R  Country or area  
11           232                I  Country or area  
12           231              B R  Country or area  
13           404              B R  Country or area  
14           450                C  Country or area

## Groups: Lists countries and corresponding major area and region

In [146]:
groups.head()

0 Country code Country or area Major area Code           Region  Code  \
0            4     Afghanistan       Asia  935    Southern Asia  5501   
1            8         Albania     Europe  908  Southern Europe   925   
2           12         Algeria     Africa  903  Northern Africa   912   
3           16  American Samoa    Oceania  909        Polynesia   957   
4           20         Andorra     Europe  908  Southern Europe   925   

0 Developed region Least developed country Sub-Saharan Africa  
0               No                     Yes                 No  
1              Yes                      No                 No  
2               No                      No                 No  
3               No                      No                 No  
4              Yes                      No                 No

# 3. Clean data tables

There are 3 types of data types:

In [147]:
# Count (e.g. population count)
df_c = []

# Percentage (e.g. male vs female)
df_p = []

# Rate (e.g. rate of change)
df_r = []

# Separate all the sheets from the UN spreadsheet
for i in list(range(1,7)):
    df = pd.read_excel("UN_MigrantStockTotal_2015.xlsx", sheet_name=i, index_col=None, skiprows=13, header=0)
    df.dropna(how='all', axis=1, inplace=True)
    # Use 'Country code' as ID between tables
    df = df.iloc[:,3:]
    # Change column names
    df.rename(columns=df.iloc[0], inplace=True)
    df.drop(df.index[0], inplace=True)
    if 'Type of data (a)' in df.columns:
        df.drop(columns='Type of data (a)', inplace=True)
    df = df.replace('..', 0)
    # Apply functions
    sub_list = []
    pos_list = []
    df = add_stat(df)
    combine_subset(df)
    # Concatenating each statistic subset into 1 dataframe
    df = pd.concat(sub_list)
    # Add table identifier
    df["Table"] = str(i)
    # Split table based on data type
    df_c.append(df[~df["Statistic"].str.contains('percentage|Annual')])
    df_p.append(df[df["Statistic"].str.contains('percentage')])
    df_r.append(df[df["Statistic"].str.contains('Annual')])
    
# Count
df_c = pd.concat(df_c)
df_c['Estimate'] = df_c['Estimate'].astype(int)
## Add classification
df_c['Classification'] = df_c['Country code']
df_c['Classification'] = df_c['Classification'].apply(add_class)

# Percentage
df_p = pd.concat(df_p)
## Add classification
df_p['Classification'] = df_p['Country code']
df_p['Classification'] = df_p['Classification'].apply(add_class)

# Rate
df_r = pd.concat(df_r)
## Add classification
df_r['Classification'] = df_r['Country code']
df_r['Classification'] = df_r['Classification'].apply(add_class)

# 4. View tables

In [149]:
df_c

Country code                                          Statistic  Year  \
0             900  International migrant stock at mid-year (both ...  1990   
1             901  International migrant stock at mid-year (both ...  1990   
2             902  International migrant stock at mid-year (both ...  1990   
3             941  International migrant stock at mid-year (both ...  1990   
4             934  International migrant stock at mid-year (both ...  1990   
...           ...                                                ...   ...   
1585          882   Estimated refugee stock at mid-year (both sexes)  2015   
1586          772   Estimated refugee stock at mid-year (both sexes)  2015   
1587          776   Estimated refugee stock at mid-year (both sexes)  2015   
1588          798   Estimated refugee stock at mid-year (both sexes)  2015   
1589          876   Estimated refugee stock at mid-year (both sexes)  2015   

       Estimate Table   Classification  
0     152563212     1        Aggregate  
1      82378628     1        Aggregate  
2      70184584     1        Aggregate  
3      11075966     1        Aggregate  
4      59105261     1        Aggregate  
...         ...   ...              ...  
1585          0     6  Country or area  
1586          0     6  Country or area  
1587          0     6  Country or area  
1588          0     6  Country or area  
1589          0     6  Country or area  

[11130 rows x 6 columns]

In [150]:
df_p

Country code                                          Statistic  Year  \
0             900  International migrant stock as a percentage of...  1990   
1             901  International migrant stock as a percentage of...  1990   
2             902  International migrant stock as a percentage of...  1990   
3             941  International migrant stock as a percentage of...  1990   
4             934  International migrant stock as a percentage of...  1990   
...           ...                                                ...   ...   
1585          882  Refugees as a percentage of the international ...  2015   
1586          772  Refugees as a percentage of the international ...  2015   
1587          776  Refugees as a percentage of the international ...  2015   
1588          798  Refugees as a percentage of the international ...  2015   
1589          876  Refugees as a percentage of the international ...  2015   

      Estimate Table   Classification  
0         2.87     3        Aggregate  
1         7.20     3        Aggregate  
2         1.69     3        Aggregate  
3         2.17     3        Aggregate  
4         1.62     3        Aggregate  
...        ...   ...              ...  
1585      0.00     6  Country or area  
1586      0.00     6  Country or area  
1587      0.00     6  Country or area  
1588      0.00     6  Country or area  
1589      0.00     6  Country or area  

[7950 rows x 6 columns]

In [151]:
df_r

Country code                                          Statistic  \
0             900  Annual rate of change of the migrant stock (bo...   
1             901  Annual rate of change of the migrant stock (bo...   
2             902  Annual rate of change of the migrant stock (bo...   
3             941  Annual rate of change of the migrant stock (bo...   
4             934  Annual rate of change of the migrant stock (bo...   
...           ...                                                ...   
1320          882         Annual rate of change of the refugee stock   
1321          772         Annual rate of change of the refugee stock   
1322          776         Annual rate of change of the refugee stock   
1323          798         Annual rate of change of the refugee stock   
1324          876         Annual rate of change of the refugee stock   

           Year  Estimate Table   Classification  
0     1990-1995      1.05     5        Aggregate  
1     1990-1995      2.28     5        Aggregate  
2     1990-1995     -0.49     5        Aggregate  
3     1990-1995      1.12     5        Aggregate  
4     1990-1995     -0.80     5        Aggregate  
...         ...       ...   ...              ...  
1320  2010-2015      0.00     6  Country or area  
1321  2010-2015      0.00     6  Country or area  
1322  2010-2015      0.00     6  Country or area  
1323  2010-2015      0.00     6  Country or area  
1324  2010-2015      0.00     6  Country or area  

[5300 rows x 6 columns]